# 다중 벡터저장소 검색기(MultiVectorRetriever)

LangChain에서는 문서를 다양한 상황에서 효율적으로 쿼리할 수 있는 특별한 기능, 바로 `MultiVectorRetriever`를 제공합니다. 이 기능을 사용하면 문서를 여러 벡터로 저장하고 관리할 수 있어, 정보 검색의 정확도와 효율성을 대폭 향상시킬 수 있습니다. 오늘은 이 `MultiVectorRetriever`를 활용해 문서당 여러 벡터를 생성하는 몇 가지 방법을 살펴보겠습니다.

**문서당 여러 벡터 생성 방법 소개**

1. **작은 청크 생성**: 문서를 더 작은 단위로 나눈 후, 각 청크에 대해 별도의 임베딩을 생성합니다. 이 방식을 사용하면 문서의 특정 부분에 좀 더 세심한 주의를 기울일 수 있습니다. 이 과정은 `ParentDocumentRetriever`를 통해 구현할 수 있어, 세부 정보에 대한 탐색이 용이해집니다.

2. **요약 임베딩**: 각 문서의 요약을 생성하고, 이 요약으로부터 임베딩을 만듭니다. 이 요약 임베딩은 문서의 핵심 내용을 신속하게 파악하는 데 큰 도움이 됩니다. 문서 전체를 분석하는 대신 핵심적인 요약 부분만을 활용하여 효율성을 극대화할 수 있습니다.

3. **가설 질문 활용**: 각 문서에 대해 적합한 가설 질문을 만들고, 이 질문에 기반한 임베딩을 생성합니다. 특정 주제나 내용에 대해 깊이 있는 탐색을 원할 때 이 방법이 유용합니다. 가설 질문은 문서의 내용을 다양한 관점에서 접근하게 해주며, 더 광범위한 이해를 가능하게 합니다.

4. **수동 추가 방식**: 사용자가 문서 검색 시 고려해야 할 특정 질문이나 쿼리를 직접 추가할 수 있습니다. 이 방법을 통해 사용자는 검색 과정에서 보다 세밀한 제어를 할 수 있으며, 자신의 요구 사항에 맞춘 맞춤형 검색이 가능해집니다.

`MultiVectorRetriever`를 통해 이러한 다양한 접근 방식을 유연하게 활용함으로써, 사용자는 필요한 정보를 보다 정확하고 빠르게 찾을 수 있습니다. LangChain의 이 기능은 정보 검색 작업을 보다 효과적으로 만들어주는 훌륭한 도구입니다.


## 사용 방법


텍스트 파일에서 데이터를 로드하고, 로드된 문서들을 지정된 크기로 분할하는 전처리 과정을 수행합니다.

분할된 문서들은 추후 벡터화 및 검색 등의 작업에 사용될 수 있습니다.


In [3]:
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

loaders = [
    # 첫 번째 데이터를 로드합니다.
    TextLoader("./data/ai-story.txt"),
    # 두 번째 데이터를 로드합니다.
    TextLoader("./data/appendix-keywords.txt"),
]
docs = []  # 빈 문서 리스트를 초기화합니다.
for loader in loaders:
    docs.extend(loader.load())  # 각 로더에서 문서를 로드하여 docs 리스트에 추가합니다.

데이터로부터 로드한 원본 도큐먼트는 `docs` 변수에 담았습니다.


## 작은 청크 생성

대용량 정보를 검색하는 경우, 더 작은 단위로 정보를 임베딩하는 것이 유용할 수 있습니다.

이를 통해 임베딩은 의미론적 의미를 최대한 근접하게 포착하면서도, 가능한 한 많은 맥락을 하위 단계로 전달할 수 있습니다.

`ParentDocumentRetriever`가 수행하는 작업이 바로 이것입니다.


In [4]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [5]:
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소
import uuid

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장소 계층
store = InMemoryByteStore()

id_key = "doc_id"
# 검색기 (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# 문서 ID를 생성합니다.
doc_ids = [str(uuid.uuid4()) for _ in docs]
# 두개의 생성된 id를 확인합니다.
doc_ids

['5e3e94ba-2f82-4b6d-a5f9-33489635ec17',
 '5447f4f3-8bc0-4ce6-80f9-8b26db08348a']

여기서 큰 청크로 분할하기 위한 `parent_text_splitter`

더 작은 청크로 분할하기 위한 `child_text_splitter` 를 정의합니다.


In [6]:
# RecursiveCharacterTextSplitter 객체를 생성합니다.
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000)

# 더 작은 청크를 생성하는 데 사용할 분할기
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

- `parent_text_splitter`를 사용하여 문서를 큰 청크 단위로 분할합니다.
- 각 문서의 메타데이터에 `"doc_id"`를 키로 하고 생성한 `uuid` 를 입력합니다.
- 최종적으로 `parent_docs` 리스트에는 원본 문서들이 큰 단위로 분할된 하위 문서들이 저장되며, 각 문서에는 원본 문서의 ID가 메타데이터로 포함됩니다.


In [7]:
parent_docs = []

for i, doc in enumerate(docs):
    _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
    # 현재 문서를 하위 문서로 분할합니다.
    parent_doc = parent_text_splitter.split_documents([doc])
    for _doc in parent_doc:  # 분할된 문서에 대해 반복합니다.
        # 문서의 메타데이터에 ID를 저장합니다.
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)  # 분할된 문서를 리스트에 추가합니다.

- `child_text_splitter`를 사용하여 문서를 더 작은 청크로 분할합니다.
- 각 문서의 메타데이터에 `"doc_id"`를 키로 하고 생성한 `uuid` 를 입력합니다. 이는 작게 분할된 청크에 문서의 ID 를 부여하기 위함입니다.
- 최종적으로 `child_docs` 리스트에는 원본 문서들이 작게 분할된 하위 문서들이 저장되며, 각 하위 문서에는 원본 문서의 ID가 메타데이터로 포함됩니다.


In [8]:
child_docs = []  # 하위 문서를 저장할 리스트를 초기화합니다.
for i, doc in enumerate(docs):
    _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
    # 현재 문서를 하위 문서로 분할합니다.
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:  # 분할된 하위 문서에 대해 반복합니다.
        # 하위 문서의 메타데이터에 ID를 저장합니다.
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)  # 분할된 하위 문서를 리스트에 추가합니다.

각각 분할된 청크의 수를 확인합니다.


In [9]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 4
분할된 child_docs의 개수: 54


벡터저장소에 새롭게 생성한 작게 쪼개진 하위문서 집합을 추가합니다.

다음으로는 상위 문서는 생성한 UUID 와 맵핑하여 `docstore` 에 추가합니다.

- `mset()` 메서드를 통해 문서 ID와 문서 내용을 key-value 쌍으로 문서 저장소에 저장합니다.

(예시) `list(zip(doc_ids, docs))[0]`

```
('36d475a5-9f1a-40ab-aeb1-ba720fa229d8',
 Document(page_content='Scikit Learn\n\nScikit-learn은 Python 언어를 위한 또 다른 핵심 라이브러리로, 기계 학습의 다양한 알고리즘을 구현하기 위해 설계되었습니다. 이 라이브러리는 2007년 David Cournapeau에 의해 프로젝트가 시작되었으며, 그 후로 커뮤니티의 광범위한 기여를 받아 현재까지 발전해왔습니다. ...))
```


In [12]:
# 벡터 저장소에 하위 문서를 추가합니다.
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

# 문서 저장소에 문서 ID와 문서를 매핑하여 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, docs)))

주어진 키워드에 대한 유사도 검색을 수행합니다. 가장 유사도가 높은 첫 번째 문서 조각을 출력합니다.


In [17]:
# vectorstore의 유사도 검색을 수행합니다.
retriever.vectorstore.similarity_search("Word2Vec 의 정의?")

[Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': '5e3e94ba-2f82-4b6d-a5f9-33489635ec17', 'source': './data/ai-story.txt'}),
 Document(page_content='Word2Vec은 크게 두 가지 모델 아키텍처로 구성됩니다: Continuous Bag-of-Words (CBOW)와 Skip-Gram입니다. CBOW 모델은 주변 단어(맥락)를 기반으로 특정 단어를 예측하는 반면, Skip-Gram 모델은 하나의 단어로부터 주변 단어들을 예측합니다. 두 모델 모두 딥러닝이 아닌, 단순화된 신경망 구조를 사용하여 대규모 텍스트 데이터에서 학습할 수 있으며, 매우 효율적입니다.', metadata={'doc_id': '5e3e94ba-2f82-4b6d-a5f9-33489635ec17', 'source': './data/ai-story.txt'}),
 Document(page_content='Word2Vec의 벡터 표현은 다양한 NLP 작업에 활용될 수 있습니다. 예를 들어, 단어의 유사도 측정, 문장이나 문서의 벡터 표현 생성, 기계 번역, 감정 분석 등이 있습니다. 또한, 벡터 연산을 통해 단어 간의 의미적 관계를 추론하는 것이 가능해집니다. 예를 들어, "king" - "man" + "woman"과 같은 벡터 연산을 수행하면, 결과적으로 "queen"과 유사한 벡터를 가진 단어를 찾을 수 있습니다.', metadata={'doc_id': '5e3e94ba-2f82-

`retriever` 객체의 `get_relevant_documents` 메서드를 호출하여 관련된 문서를 검색합니다.

주어진 쿼리에 관련성이 높은 문서를 검색합니다.

여기서는 2개의 도큐먼트 안에 "Word2Vec" 의 정의가 포함되었기 때문에 2개의 문서 모두 검색 결과로 나왔습니다.


In [20]:
relevant_doc = retriever.get_relevant_documents("Word2Vec 의 정의?")
len(relevant_doc)

2

첫 번째로 찾은 문서의 내용의 문자열 길이를 확인하면 문서 전체가 출력됨을 확인할 수 있습니다.


In [21]:
# Retriever가 반환하는 문서의 길이를 확인합니다.
len(retriever.get_relevant_documents("Word2Vec 의 정의?")[0].page_content)

7482

리트리버(retriever)가 벡터 데이터베이스에서 기본적으로 수행하는 검색 유형은 유사도 검색입니다.

LangChain Vector Stores는 [Max Marginal Relevance](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search)를 통한 검색도 지원하므로, 이를 대신 사용하고 싶다면 다음과 같이 `search_type` 속성을 설정하면 됩니다.


- `retriever` 객체의 `search_type` 속성을 `SearchType.mmr`로 설정합니다.
  - 이는 검색 시 MMR(Maximal Marginal Relevance) 알고리즘을 사용하도록 지정하는 것입니다.


In [24]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 MMR(Maximal Marginal Relevance)로 설정
retriever.search_type = SearchType.mmr

# 검색어로 관련 문서를 검색하고, 첫 번째 문서의 페이지 내용 길이를 반환
len(retriever.get_relevant_documents("Word2Vec의 정의")[0].page_content)

7482

## 요약본(summary)을 벡터저장소에 저장

요약은 종종 청크(chunk)의 내용을 보다 정확하게 추출할 수 있어 더 나은 검색 결과를 얻을 수 있습니다.

여기서는 요약을 생성하는 방법과 이를 임베딩하는 방법에 대해 설명합니다.


In [25]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


chain = (
    {"doc": lambda x: x.page_content}  # 입력 데이터에서 페이지 내용을 추출하는 함수
    # 문서 요약을 위한 프롬프트 템플릿 생성
    | ChatPromptTemplate.from_template(
        "Summarize the following document in Korean:\n\n{doc}"
    )
    # OpenAI의 ChatGPT 모델을 사용하여 요약 생성 (최대 재시도 횟수: 0)
    | ChatOpenAI(max_retries=0)
    | StrOutputParser()  # 생성된 요약 결과를 문자열로 파싱
)

- `chain.batch` 메서드를 사용하여 `docs` 리스트의 문서들을 일괄 처리합니다.
- 여기서 `max_concurrency` 매개변수를 5로 설정하여 최대 5개의 문서를 동시에 처리할 수 있도록 합니다.


In [26]:
# 최대 동시성을 5로 설정하여 문서 배치 처리
summaries = chain.batch(docs, {"max_concurrency": 5})

요약된 내용을 출력하여 결과를 확인합니다.


In [27]:
# 요약을 출력합니다.
print(summaries[0])

Scikit-learn은 Python을 위한 핵심 머신 러닝 라이브러리로, 다양한 알고리즘을 구현하기 위해 설계되었으며 분류, 회귀, 군집화, 차원 축소 등 다양한 머신 러닝 작업을 지원합니다. NLP는 컴퓨터 알고리즘과 기술의 집합으로, 자연어를 처리하여 사람과 컴퓨터 간의 상호작용을 향상시킵니다. SciPy는 과학 계산을 위한 중심적인 파이썬 라이브러리로, 수학 함수, 알고리즘, 도구를 제공하며, Hugging Face는 NLP 분야에서 선도적인 역할을 하는 기술 회사로, 다양한 오픈 소스 도구와 라이브러리를 제공합니다. "Attention Is All You Need"는 변환기 모델의 아키텍처를 소개한 연구로, NLP 분야에 큰 영향을 미쳤습니다. ARIMA, SARIMA, SARIMAX는 시계열 데이터 분석과 예측을 위한 통계 모델이며, Word2Vec은 단어 임베딩 기법 중 하나로, 단어의 의미를 수치적으로 이해할 수 있게 합니다.


In [28]:
# 요약을 출력합니다.
print(summaries[1])

의미론적 검색은 사용자의 질의를 키워드 매칭을 넘어 의미를 파악하여 관련된 결과를 반환하는 방식이다. 임베딩은 텍스트 데이터를 벡터로 변환하는 과정이며, 토큰은 텍스트를 작은 단위로 분할하는 것을 의미한다. 토크나이저는 텍스트를 토큰으로 분할하는 도구이다. 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템이다. SQL은 데이터베이스를 관리하기 위한 프로그래밍 언어이다. CSV는 데이터를 저장하는 파일 형식으로, JSON은 데이터 교환 형식이다. 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델이며, HuggingFace는 NLP 작업을 위한 라이브러리이다. 디지털 변환은 기술을 활용하여 기업을 혁신하는 과정이며, 크롤링은 웹 페이지에서 데이터를 수집하는 과정이다. Word2Vec은 자연어 처리 기술로, LLM은 대규모 언어 모델을 의미한다. FAISS는 유사성 검색 라이브러리이고, 오픈 소스는 소프트웨어 개발을 위한 개방적인 접근 방식이다. 구조화된 데이터는 정해진 형식에 따라 조직된 데이터이고, 파서는 데이터를 구조화된 형태로 변환하는 도구이다. TF-IDF는 단어의 중요도를 평가하는 통계적 척도이다. 딥러닝은 복잡한 문제를 해결하는 머신러닝 분야이며, 스키마는 데이터의 구조를 정의하는 데 사용된다. DataFrame은 데이터 테이블 구조이고, Attention은 중요한 정보에 집중하는 기법이다. 판다스는 데이터 분석 도구이고, GPT는 생성적 언어 모델이다. InstructGPT는 특정 작업을 수행하기 위한 GPT 모델이다. 키워드 검색은 정보를 찾는 과정이며, 페이지 랭크는 웹 페이지 중요도를 평가하는 알고리즘이다. 데이터 마이닝은 유용한 정보를 발굴하는 과정이고, 멀티모달은 여러 종류의 데이터 모드를 처리하는 기술이다.


`Chroma` 벡터 저장소를 초기화하여 자식 청크(child chunks)를 인덱싱합니다. 이때 `OpenAIEmbeddings`를 임베딩 함수로 사용합니다.

- 문서 ID를 나타내는 키로 `"doc_id"`를 사용합니다.


In [63]:
# 요약 정보를 저장할 벡터 저장소를 생성합니다.
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=OpenAIEmbeddings())
# 부모 문서를 저장할 저장소를 생성합니다.
store = InMemoryByteStore()
# 문서 ID를 저장할 키 이름을 지정합니다.
id_key = "doc_id"
# 검색기를 초기화합니다. (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,  # 벡터 저장소
    byte_store=store,  # 바이트 저장소
    id_key=id_key,  # 문서 ID 키
)
# 문서 ID를 생성합니다.
doc_ids = [str(uuid.uuid4()) for _ in docs]

요약된 문서와 메타데이터(여기서는 생성한 요약본에 대한 `Document ID` 입니다)를 저장합니다.


In [64]:
summary_docs = [
    # 요약된 내용을 페이지 콘텐츠로 하고, 문서 ID를 메타데이터로 포함하는 Document 객체를 생성합니다.
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(
        summaries
    )  # summaries 리스트의 각 요약과 인덱스에 대해 반복합니다.
]

요약본의 문서의 개수는 원본 문서의 개수와 일치합니다.


In [65]:
# 요약본의 문서의 개수
len(summary_docs)

2

- `retriever.vectorstore.add_documents(summary_docs)`를 통해 `summary_docs`를 벡터 저장소에 추가합니다.
- `retriever.docstore.mset(list(zip(doc_ids, docs)))`를 사용하여 `doc_ids`와 `docs`를 매핑하여 문서 저장소에 저장합니다.


In [66]:
retriever.vectorstore.add_documents(
    summary_docs
)  # 요약된 문서를 벡터 저장소에 추가합니다.

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, docs)))

다음으로는 원본 청크(chunk) 데이터를 벡터 저장소에 추가하는 코드입니다.


In [67]:
# RecursiveCharacterTextSplitter 객체를 생성합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

split_docs = []
split_docs_ids = []

for i, doc in enumerate(docs):
    _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
    # 현재 문서를 하위 문서로 분할합니다.
    split_doc = text_splitter.split_documents([doc])
    for _doc in split_doc:  # 분할된 문서에 대해 반복합니다.
        # 문서의 메타데이터에 ID를 저장합니다.
        _doc.metadata[id_key] = _id
        split_docs_ids.append(_id)
    split_docs.extend(split_doc)  # 분할된 문서를 리스트에 추가합니다.

분할된 문서의 개수를 출력합니다.


In [68]:
print(f"분할된 문서의 개수: {len(split_docs)}")

분할된 문서의 개수: 17


마지막으로 분할된 문서를 벡터 저장소에 추가합니다.


In [69]:
# 문서를 검색기의 벡터 저장소에 추가합니다.
retriever.vectorstore.add_documents(split_docs)

['8e0bc136-b2aa-4447-bbc7-7fb079c706f7',
 'f785754d-9881-4000-a876-03087694d3f1',
 '792d2d6d-6e64-4746-8261-0d737e7a6a11',
 '112bdce8-4e2b-4e63-83bb-c539061c0eb3',
 '7c7b7107-63c9-4c65-96e7-f55c80f90570',
 'cda29ed6-1f73-4a34-a6cc-a6bc558aaf12',
 'b1600491-0531-4dfe-a506-825bf3a659db',
 'b3258b44-8393-4a66-860d-e273729e2e2e',
 'b7fe87e7-b42c-44a2-bf34-267d21f6f702',
 '16e0a7a9-f43a-4f08-8986-36ff75dc0b55',
 '48256130-6d67-43e9-9e54-2d9949da773f',
 '9db331b3-7606-4d5e-98ef-5d53978113f7',
 '0b6cd52f-58ab-42d1-8c19-5039a4567196',
 '34a9f550-7c13-4e52-b45b-b263bcadc25d',
 'cfcc2d6c-9e77-4cb9-95a0-62c93fd234f4',
 '220dab59-8302-46fd-acdf-fa0d69fc1d53',
 'daec8d42-83f9-4692-8192-48f73337cd5d']

`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [78]:
# 유사도 검색을 수행합니다.
result_docs = vectorstore.similarity_search("Word2Vec의 정의가 뭐야?")

In [88]:
# 1개의 결과 문서를 출력합니다.
result_docs[0]

Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': 'fb470c39-310e-4687-9776-705d52d1fd3f', 'source': './data/ai-story.txt'})

`retriever` 객체의 `get_relevant_documents` 메서드를 사용하여 질문과 관련된 문서를 검색합니다.


In [85]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents("Word2Vec 의 정의가 뭐야?")
len(retrieved_docs)

1

`retrieved_docs[0].page_content`의 길이를 반환합니다. 문서를 반환하기 때문에 페이지 내용의 길이는 청크보다 일반적으로 큽니다.


In [86]:
# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
len(retrieved_docs[0].page_content)

7482

## 가설 쿼리(Hypothetical Queries)

LLM은 특정 문서에 대해 가정할 수 있는 질문 목록을 생성하는 데에도 사용될 수 있습니다.

이렇게 생성된 질문들은 임베딩(embedding)될 수 있으며, 이를 통해 문서의 내용을 더욱 깊이 있게 탐색하고 이해할 수 있습니다.

가정 질문 생성은 문서의 주요 주제와 개념을 파악하는 데 도움이 되며, 독자들이 문서 내용에 대해 더 많은 궁금증을 갖도록 유도할 수 있습니다.


- `functions` 리스트에는 하나의 딕셔너리 요소가 포함되어 있습니다.
- 딕셔너리는 `name`, `description`, `parameters` 키를 가지고 있습니다.
- `name`은 함수의 이름을 나타내는 문자열입니다.
- `description`은 함수의 설명을 나타내는 문자열입니다.
- `parameters`는 함수의 매개변수를 정의하는 딕셔너리입니다.
  - `type`은 매개변수의 타입을 나타내며, 여기서는 "object"로 설정되어 있습니다.
  - `properties`는 객체의 속성을 정의하는 딕셔너리입니다.
    - `questions`는 "array" 타입의 속성으로, 각 요소는 "string" 타입입니다.
  - `required`는 필수 속성을 나타내는 리스트이며, 여기서는 `questions`가 필수로 지정되어 있습니다.


In [89]:
functions = [
    {
        "name": "hypothetical_questions",  # 함수의 이름을 지정합니다.
        "description": "Generate hypothetical questions",  # 함수에 대한 설명을 작성합니다.
        "parameters": {  # 함수의 매개변수를 정의합니다.
            "type": "object",  # 매개변수의 타입을 객체로 지정합니다.
            "properties": {  # 객체의 속성을 정의합니다.
                "questions": {  # 'questions' 속성을 정의합니다.
                    "type": "array",  # 'questions'의 타입을 배열로 지정합니다.
                    "items": {
                        "type": "string"
                    },  # 배열의 요소 타입을 문자열로 지정합니다.
                },
            },
            "required": ["questions"],  # 필수 매개변수로 'questions'를 지정합니다.
        },
    }
]

`ChatPromptTemplate`을 사용하여 주어진 문서를 기반으로 3개의 가상 질문을 생성하는 프롬프트 템플릿을 정의합니다.

- `ChatOpenAI`를 사용하여 GPT 모델을 초기화하고, `functions`와 `function_call`을 설정하여 가상 질문 생성 함수를 호출합니다.
- `JsonKeyOutputFunctionsParser`를 사용하여 생성된 가상 질문을 파싱하고, `questions` 키에 해당하는 값을 추출합니다.


In [90]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    {"doc": lambda x: x.page_content}
    # 아래 문서를 사용하여 답변할 수 있는 가상의 질문을 정확히 3개 생성하도록 요청합니다. 이 숫자는 조정될 수 있습니다.
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer. Answer in Korean:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4-turbo-preview").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    # 출력에서 "questions" 키에 해당하는 값을 추출합니다.
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

`chain.invoke(docs[0])` 호출하여 첫 번째 문서에 대한 답변을 출력합니다.

- 출력은 생성한 3개의 가설 쿼리(Hypothetical Queries) 가 담겨 있습니다.


In [91]:
# 주어진 문서에 대해 체인을 실행합니다.
chain.invoke(docs[0])

['Scikit-learn의 핵심 특징은 무엇이며, 이로 인해 사용자가 어떤 이점을 얻을 수 있나요?',
 'NLP의 핵심 과제는 무엇이며, 이를 해결하기 위한 기술적 방법은 무엇인가요?',
 'Word2Vec의 두 가지 모델 아키텍처는 각각 어떤 작업을 수행하며, 이러한 모델들이 어떻게 단어 간의 의미적 관계를 벡터 연산을 통해 표현할 수 있나요?']

`chain.batch` 메서드를 사용하여 `docs` 데이터에 대해 동시에 여러 개의 요청을 처리합니다.

- `docs` 매개변수는 처리할 문서 데이터를 나타냅니다.
- `max_concurrency` 매개변수는 동시에 처리할 수 있는 최대 요청 수를 지정합니다. 이 예시에서는 5로 설정되어 있습니다.
- 이 메서드는 `docs` 데이터의 각 항목에 대해 `chain` 객체의 작업을 수행하고, 최대 5개의 요청을 동시에 처리합니다.
- 처리 결과는 `hypothetical_questions` 변수에 저장됩니다.


In [92]:
# 문서 목록에 대해 가설적 질문을 일괄 처리하여 생성합니다. 최대 동시성은 5로 설정되어 있습니다.
hypothetical_questions = chain.batch(docs, {"max_concurrency": 5})

In [93]:
print(hypothetical_questions[0])
print(hypothetical_questions[1])

['Scikit-learn을 사용하여 어떤 기계 학습 모델을 개발할 수 있을까요?', 'NLP 기술을 활용하여 어떤 문제를 해결할 수 있을까요?', 'SciPy 라이브러리를 이용한 과학 계산에서 어떤 종류의 문제를 해결할 수 있을까요?']
['자연어 처리를 통한 의미론적 검색의 장점은 무엇인가요?', '텍스트 데이터를 벡터로 변환하는 임베딩 과정은 어떻게 이루어지나요?', '토큰화 과정을 통해 자연어 처리의 성능을 어떻게 향상시킬 수 있나요?']


아래는 이전에 진행했던 방식과 동일하게 생성한 가설 쿼리(Hypothetical Queries) 를 벡터저장소에 저장하는 과정입니다.


In [127]:
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소
vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장소 계층
store = InMemoryByteStore()
id_key = "doc_id"
# 검색기 (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]  # 문서 ID 생성

`question_docs` 리스트에 메타데이터(문서 ID) 를 추가합니다.


In [128]:
question_docs = []
# hypothetical_questions 리스트를 순회하면서 인덱스와 질문 리스트를 가져옵니다.
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(  # question_docs 리스트에 Document 객체를 추가합니다.
        # 질문 리스트의 각 질문에 대해 Document 객체를 생성하고, 메타데이터에 해당 질문의 문서 ID를 포함시킵니다.
        [Document(page_content=s, metadata={
                  id_key: doc_ids[i]}) for s in question_list]
    )

가설 쿼리를 문서에 추가하고, 원본 문서를 `docstore` 에 추가합니다.


In [129]:
retriever.vectorstore.add_documents(
    question_docs
)  # 질문 문서를 벡터 저장소에 추가합니다.
# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, docs)))

`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [130]:
# 유사한 문서를 벡터 저장소에서 검색합니다.
result_docs = vectorstore.similarity_search("Word2Vec에 대한 정의는 뭐야?")

아래는 유사도 검색 결과입니다.

여기서는 생성한 가설 쿼리만 추가해 놓은 상태이기 때문에, 생성한 가설 쿼리 중 유사도가 가장 높은 문서를 반환합니다.


In [131]:
# 유사도 검색 결과를 출력합니다.
result_docs

[Document(page_content='Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.', metadata={'doc_id': '321a362f-ea00-4703-9513-d7ca3e94c7ae', 'source': './data/ai-story.txt'}),
 Document(page_content='텍스트 데이터를 벡터로 변환하는 임베딩 과정은 어떻게 이루어지나요?', metadata={'doc_id': '011de84d-4fe8-4344-960f-402a0174b820'}),
 Document(page_content='텍스트 데이터를 벡터로 변환하는 임베딩 과정은 어떻게 이루어지나요?', metadata={'doc_id': '9630bc35-f039-4117-8388-54d1d97feb8b'}),
 Document(page_content='텍스트 데이터를 벡터로 변환하는 임베딩 과정은 어떻게 이루어지나요?', metadata={'doc_id': 'b9d9392b-250e-4a5f-999d-c2302182d773'})]

이전 단계에서 분할한 문서도 벡터저장소에 추가합니다.


In [132]:
# 문서를 검색기의 벡터 저장소에 추가합니다.
retriever.vectorstore.add_documents(split_docs)

['907d1b95-8754-4c4a-ad32-c921fa33aa49',
 '07f6e09d-684c-433d-a503-f2d8b734089e',
 '926648d6-c4c2-495c-ba10-383d5fa83188',
 'edb4b2dd-c5bd-4259-8756-0ae9e4468085',
 '15e63c9f-c692-4b58-8e58-bd10b69aa721',
 '77e7aef3-de81-43d8-a604-759e09a52bcf',
 'cd9939fe-77d8-4962-b4bf-9fbc04b42e89',
 'da257e5a-417a-48a6-b397-559b1dc4b323',
 'ff274756-2761-4cd6-bc12-ce37b4d18e80',
 'f6145d36-456c-450c-b658-32c47da758c5',
 'e7e2c935-3081-4a6c-801e-d6875a92ab91',
 'c23c2c89-16fa-4b33-b474-5e4556a6d1ce',
 'e0a6356f-dd33-4530-a4fd-a31cbcc3b7af',
 '25a7ed32-bc05-4457-b2b8-e7c6535d355d',
 'a1c73bf3-f3a4-468f-bf04-c809230af996',
 '07af82e4-bffc-4c11-a713-5d1e329aa121',
 'f1505bdd-57e4-4942-a42b-279f2678e095']

`retriever` 객체의 `get_relevant_documents` 메서드를 사용하여 쿼리와 관련된 문서를 검색합니다.


In [133]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents("Word2Vec에 대한 정의가 뭐야?")
len(retrieved_docs)

1

검색 결과인 `retrieved_docs[0].page_content`의 길이를 확인합니다.


In [134]:
# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
len(retrieved_docs[0].page_content)

5733